# **Niche Recommender System for Movies** ![Neural Nets](https://i.imgur.com/aa0fVPE.jpg1!)


In [ ]:
%tensorflow_version 2.x
from __future__ import absolute_import, division, print_function, unicode_literals
print("Run on TensorFlow 2.x")

Run on TensorFlow 2.x


# INITIAL STAGE

IMPORT LIBRARIES

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from sklearn.manifold import LocallyLinearEmbedding
from sklearn.model_selection import KFold

# The following lines adjust the granularity of reporting. 
pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format

# The following line improves formatting when ouputting NumPy arrays.
np.set_printoptions(linewidth = 200)

print("Imported modules.")

Using TensorFlow backend.


Imported modules.


INITIALIZE EAGER EXECUTION

In [ ]:
# After eager execution is enabled, operations are executed as they are defined 
# and Tensor objects hold concrete values.

tf.compat.v1.enable_eager_execution(
    config=None, device_policy=None, execution_mode=None
)
print("Eager Execution Enabled")

Eager Execution Enabled


LOADING DATA

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

# Pre-processing of the data was done done in R, resulting in the following 
# training and test sets, which have been split randomly with 70/30 ratio:

train_df = pd.read_csv("https://drive.google.com/file/d/1OfkihhKVTLVK_CGHFRj3ZzH63ty-xn0-/view?usp=sharing", sep='delimiter', header=None)
test_df = pd.read_csv("https://drive.google.com/file/d/1fTouaikZF3OMX1VTT77PmIXfNhBSjOpo/view?usp=sharing", sep='delimiter', header=None)

DEFINE A PLOTTING FUNCTION

In [ ]:
def plot_curve(epochs, hist, list_of_metrics):
  plt.figure()
  plt.xlabel("Epoch")
  plt.ylabel("Value")

  for m in list_of_metrics:
    x = hist[m]
    plt.plot(epochs[1:], x[1:], label=m)

  plt.legend()

print("Loaded the plot_curve function.")

Loaded the plot_curve function.


# DEFINE THE MODEL

DEEP NEURAL NET TOPOGRAPHY

In [ ]:
def create_model(my_learning_rate):
  # Destroys the current TF graph and session, and creates a new one.
    tf.keras.backend.clear_session()
  
  # Initialize sequential model.
    model = tf.keras.models.Sequential()

  # Define the first hidden layer.   
  model.add(tf.keras.layers.Dense(units=200, activation='relu'))
    
  # Define a dropout regularization layer. 
  model.add(tf.keras.layers.Dropout(rate=0.3))

  # Define the second hidden layer.   
  model.add(tf.keras.layers.Dense(units=200, activation='relu'))

  # Define a dropout regularization layer. 
  model.add(tf.keras.layers.Dropout(rate=0.3))

  # Define the output layer. The units parameter is set to 10 because
  # the model must choose among 10 possible output values (representing
  # users' ratings from 0.5 to 5, inclusive).
  model.add(tf.keras.layers.Dense(units=10, activation='softmax'))     
                           
  # Construct the layers into a model that TensorFlow can execute.  
  model.compile(optimizer=tf.keras.optimizers.Adam(lr=my_learning_rate),
                loss="sparse_categorical_crossentropy",
                metrics=['accuracy'])
  
  return model    


def train_model(model, train_features, train_label, epochs,
                batch_size=None, validation_split=0.2):
  """Train the model by feeding it data."""

  history = model.fit(x=train_features, y=train_label, batch_size=batch_size,
                      epochs=epochs, shuffle=True, 
                      validation_split=validation_split)
 
  # Tracking the progression of training.
  epochs = history.epoch
  hist = pd.DataFrame(history.history)

  return epochs, hist    

IndentationError: ignored

# BUILD AND TRAIN THE MODEL

In [ ]:
# Tuning hyperparameters.
learning_rate = 0.001
epochs = 200
batch_size = 1000

# Specify the label.
label_name = "ratings"

# Establish the model's topography.
my_model = create_model(learning_rate, my_feature_layer)

# Train the model on the training set.
epochs, mse = train_model(my_model, train_df_norm, epochs, 
                          label_name, batch_size)
plot_the_loss_curve(epochs, mse)

# After building a model against the training set, testing the model
# against the test set.
test_features = {name:np.array(value) for name, value in test_df_norm.items()}
test_label = np.array(test_features.pop(label_name))
print("\n Evaluate the new model against the test set:")
my_model.evaluate(x = test_features, y = test_label, batch_size=batch_size)

# MEASURING PERFORMANCE

TRAINING AND VALIDATION LOSS

In [ ]:
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("model loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.legend(["train", "test"], loc="upper left")
plt.show()